In [1]:
suppressMessages(library(lubridate))
suppressMessages(library(dplyr))

In [2]:
setwd("/home/aguirre/Documentos/TemasDeComputacion")
cfr_df = read.csv("coronavirus-cfr.csv")

In [3]:
cfr_df = read.csv("coronavirus-cfr.csv")
cfr_df2 <- cfr_df %>% mutate(date = Date %>% mdy()) %>% select(-Date)
diabetes_df = read.csv("diabetes-prevalence.csv")
diabetes_df2 <- diabetes_df %>% select(-Year)
bmi_df = read.csv("NCD_RisC_Lancet_2017_BMI_age_standardised_country.csv")
bmi_df2 <- bmi_df %>% filter(Year == "2016") %>% group_by(Entity, Code) %>% summarize(Mean_bmi = mean(Mean_BMI))
bloodp_df <- read.csv("NCD_RisC_Lancet_2016_BP_age_standardised_countries.csv")
bp <-bloodp_df %>% mutate( mean_bloodPressure = as.numeric(Mean_systolic_blood_pressure_.mmHg.))
bloodp_df2 <- na.omit(bp) %>% filter(Year == "2015") %>% group_by(Entity, Code) %>% summarize(Mean_bp = mean(mean_bloodPressure))
totalTest_df = read.csv("full-list-cumulative-total-tests-per-thousand.csv")
totalTest_series <- totalTest_df  %>% mutate(date = Date %>% mdy()) %>% select(-Date)
df <- inner_join(cfr_df2, totalTest_series)
total_deaths_per_millon <- read.csv("total-covid-deaths-per-million.csv") %>% rename(total_deaths_per_millon = Total.confirmed.deaths.due.to.COVID.19.per.million.people..deaths.per.million.) %>% mutate(date = Date %>% mdy()) %>% select(-Date)
median_age <- read.csv("case-fatality-rate-of-covid-19-vs-median-age.csv") %>% group_by(Entity, Code) %>% summarise(median.age = mean(Median.Age))
gdp_per_capita <- read.csv("total-confirmed-cases-of-covid-19-per-million-people-vs-gdp-per-capita.csv")  %>% mutate(date = Date %>% mdy()) %>% select(-Date, -Year) %>% rename(cases.per.million = Confirmed.COVID.19.cases.per.million.people..cases.per.million., GDP.per.capita = GDP.per.capita..international...)
stringency <- read.csv("covid-stringency-index.csv") %>% rename(Government.Response.Stringency.Index = Government.Response.Stringency.Index...0.to.100..100...strictest..) %>% mutate(date = Date %>% mdy()) %>% select(-Date)
population <- read.csv("datasets_507962_1091873_population_by_country_2020.csv") %>% rename(Entity = Country..or.dependency., Population2020 = Population..2020., Density_P_Km =Density..P.Km.. ) %>% select(Entity, Population2020, Density_P_Km)

`summarise()` regrouping output by 'Entity' (override with `.groups` argument)
`summarise()` regrouping output by 'Entity' (override with `.groups` argument)
Joining, by = c("Entity", "Code", "date")
`summarise()` regrouping output by 'Entity' (override with `.groups` argument)


In [4]:
df1<- inner_join( df, diabetes_df2)
df2 <- inner_join( df1, bmi_df2)  
df3 <- inner_join( df2, bloodp_df2)
df4 <- inner_join(df3, totalTest_series)
df5 <- inner_join(df4, total_deaths_per_millon)
df6 <- inner_join(df5, median_age)
df7 <- inner_join(df6, gdp_per_capita)
df8 <- inner_join(df7, stringency)
df9 <- inner_join(df8, population)

Joining, by = c("Entity", "Code")
Joining, by = c("Entity", "Code")
Joining, by = c("Entity", "Code")
Joining, by = c("Entity", "Code", "date", "Total_tests_per_thousand")
Joining, by = c("Entity", "Code", "date")
Joining, by = c("Entity", "Code")
Joining, by = c("Entity", "Code", "date")
Joining, by = c("Entity", "Code", "date")
Joining, by = "Entity"


In [7]:
df_final <- df9 %>% filter(CFR > 0) %>% group_by(Entity, Code) %>% mutate(indexD = seq(as.Date("2020-01-01"), by = "day", length.out = length(Entity))) %>% group_by(Entity, Code, week = week(indexD)) %>% summarise(CFR = mean(CFR), Total_tests_per_thousand = mean(Total_tests_per_thousand), Diabetes_prevalence = mean(Diabetes_prevalence), Mean_bmi = mean(Mean_bmi), Mean_bp = mean(Mean_bp), total_deaths_per_millon = mean(total_deaths_per_millon), median.age = mean(median.age), cases.per.million = mean(cases.per.million), GDP.per.capita = mean(GDP.per.capita),  Government.Response.Stringency.Index = mean(Government.Response.Stringency.Index), Population2020 =mean(Population2020), Density_P_Km = mean(Density_P_Km))

`summarise()` regrouping output by 'Entity', 'Code' (override with `.groups` argument)


In [8]:
df_final

Entity,Code,week,CFR,Total_tests_per_thousand,Diabetes_prevalence,Mean_bmi,Mean_bp,total_deaths_per_millon,median.age,cases.per.million,GDP.per.capita,Government.Response.Stringency.Index,Population2020,Density_P_Km
Argentina,ARG,1,4.0550000,0.4094286,5.50,27.90814,124.4267,1.8458571,46.9,45.15586,18933.91,100.00000,45111229,17
Argentina,ARG,2,4.5644286,0.7032857,5.50,27.90814,124.4267,1.5108571,46.9,42.38686,18933.91,74.60286,45111229,17
Argentina,ARG,3,4.6268571,0.7861429,5.50,27.90814,124.4267,2.6204286,46.9,47.23571,18933.91,74.60286,45111229,17
Argentina,ARG,4,4.9100000,1.0914286,5.50,27.90814,124.4267,4.0015714,46.9,81.42343,18933.91,93.65286,45111229,17
Argentina,ARG,5,5.1095714,1.4534286,5.50,27.90814,124.4267,5.2595714,46.9,102.80671,18933.91,85.19000,45111229,17
Argentina,ARG,6,5.1554286,1.8334286,5.50,27.90814,124.4267,6.5492857,46.9,127.25900,18933.91,87.56857,45111229,17
Argentina,ARG,7,4.6870000,2.2908571,5.50,27.90814,124.4267,8.0697143,46.9,NA,18933.91,79.36429,45111229,17
Argentina,ARG,8,4.0700000,2.7757143,5.50,27.90814,124.4267,9.5807143,46.9,238.06271,18933.91,79.36429,45111229,17
Argentina,ARG,9,3.5920000,3.2100000,5.50,27.90814,124.4267,11.0630000,46.9,307.99300,18933.91,90.74000,45111229,17
Australia,AUS,1,0.6474286,10.7244286,5.07,27.29579,119.6601,1.2774286,46.7,191.54157,44648.71,68.78429,25439164,3
